In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# 학습 파일 불러오기
def read_file(file_name) :
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

In [3]:
# 학습용 말뭉치 데이터 불러오기
corpus = read_file('KoreanNERCorpus/modified/train.txt')

# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])

    sentences.append(sentence)
    tags.append(bio_tag)

print("샘플 크기 : \n", len(sentences))

샘플 크기 : 
 3555


In [4]:
print("0번째 샘플 문장 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 : \n", tags[0])
print("샘플 문장 시퀀스 최대 길이 : ", max(len(l) for l in sentences))
print("샘플 문장 시퀀스 평균 길이 : ", (sum(map(len, sentences))/len(sentences)))

0번째 샘플 문장 시퀀스 : 
 ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그 : 
 ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이 :  168
샘플 문장 시퀀스 평균 길이 :  34.03909985935302


In [5]:
# 토크나이저 정의
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='00V')      # 첫번째 인덱스에는 00V 사용
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False)           # 태그 정보는 lower=False 소문자로 변환하지 않는다
tag_tokenizer.fit_on_texts(tags)

# 단어 사전 및 태그 사전 크기
vocab_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 : ", tag_size)
print("단어 사전 크기 : ", vocab_size)

BIO 태그 사전 크기 :  8
단어 사전 크기 :  13834


In [6]:
# 학습용 단어 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[0])
print(y_train[0])

[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3]
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [7]:
# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word       # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word         # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = 'PAD'

# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

# 학습 데이터와 테스트 데이터를 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=0)

In [8]:
# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (2844, 40)
학습 샘플 레이블 형상 :  (2844, 40, 8)
테스트 샘플 시퀀스 형상 :  (711, 40)
테스트 샘플 레이블 형상 :  (711, 40, 8)


In [9]:
# 모델 정의(Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)

print("평가 결과 : ", model.evaluate(x_test, y_test)[1])

Epoch 1/10
23/23 [==============================] - 11s 352ms/step - loss: 0.6991 - accuracy: 0.8371
Epoch 2/10
23/23 [==============================] - 9s 393ms/step - loss: 0.3140 - accuracy: 0.9002
Epoch 3/10
23/23 [==============================] - 10s 420ms/step - loss: 0.2225 - accuracy: 0.9237
Epoch 4/10
23/23 [==============================] - 11s 465ms/step - loss: 0.1683 - accuracy: 0.9429
Epoch 5/10
23/23 [==============================] - 11s 456ms/step - loss: 0.1272 - accuracy: 0.9586
Epoch 6/10
23/23 [==============================] - 11s 458ms/step - loss: 0.0894 - accuracy: 0.9727
Epoch 7/10
23/23 [==============================] - 10s 449ms/step - loss: 0.0642 - accuracy: 0.9803
Epoch 8/10
23/23 [==============================] - 10s 446ms/step - loss: 0.0541 - accuracy: 0.9829
Epoch 9/10
23/23 [==============================] - 11s 457ms/step - loss: 0.0411 - accuracy: 0.9871
Epoch 10/10
23/23 [==============================] - 2s 79ms/step - loss: 0.2925 - accuracy:

In [10]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences) :
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace("PAD", "0"))
        result.append(temp)
    return result

In [13]:
# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test)
pred_tags = sequences_to_tag(y_predicted)       # 예측된 NER
test_tags = sequences_to_tag(y_test)            # 실제 NER

# F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

23/23 [==============================] - 1s 62ms/step


c:\Users\User\anaconda3\envs\KoNLPy\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\User\anaconda3\envs\KoNLPy\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\User\anaconda3\envs\KoNLPy\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\User\anaconda3\envs\KoNLPy\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\User\anaconda3\envs\KoNLPy\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\User

              precision    recall  f1-score   support

           _       0.58      0.58      0.58       644
         _DT       0.93      0.89      0.91       335
         _LC       0.69      0.55      0.61       312
         _OG       0.70      0.53      0.61       481
         _PS       0.77      0.42      0.54       374
         _TI       0.80      0.77      0.78        66

   micro avg       0.71      0.59      0.64      2212
   macro avg       0.74      0.62      0.67      2212
weighted avg       0.71      0.59      0.64      2212

F1-score: 64.4%


In [18]:
# 새로운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
new_input = str(input("새로운 문장 입력 : "))
new_sentence = new_input.split()
new_x = []
for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w, 1))
    except KeyError:
        # 모르는 단어의 경우 00V
        new_x.append(word_to_index['00V'])

print("새로운 유형의 시퀀스 : ", new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding='post', value=0, maxlen=max_len)

# NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=-1)

print("{:10} {:5}".format("단어", "예측된 NER"))
print("-" * 50)
for w, pred in zip(new_sentence, p[0]):
    print("{:10} {:5}".format(w, index_to_ner[pred]))

새로운 유형의 시퀀스 :  [531, 286, 1, 1476, 1, 6766, 1]
1/1 [==============================] - 0s 20ms/step
단어         예측된 NER
--------------------------------------------------
삼성전자       B_OG 
오늘         B_DT 
출시한        O    
스마트폰       O    
애플에        O    
도전장        O    
내밀다        O    
